In [14]:
import numpy as np
import tifffile
import matplotlib.pyplot as plt
import os
import glob
import keras
from keras.utils import np_utils
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam

In [15]:
folder = "D:\BScope\\SequentialBeads\\stacks\\TrainingData\\"

In [16]:
def onehot_encode(arr):
    
    uniques, ids = np.unique(arr, return_inverse=True)
    return np_utils.to_categorical(ids, len(uniques))

In [17]:
# Label encoding
classes = os.listdir(folder)
onehot = onehot_encode(classes)

for i,c in enumerate(classes):
    print('%10s → [%d,%d,%d]' % (c, onehot[i,0], onehot[i,1], onehot[i,2]))

   Defocus → [1,0,0]
ObliqueAstigmation → [0,1,0]
VerticalAstigmation → [0,0,1]


In [18]:
# Loading DataSet
imageData = list();
modes = list();
subDir = os.listdir(folder)

for s in subDir:
    files  = glob.glob(folder+ s +"\\*.tif" )
    for f in files:
        imageData.append(tifffile.imread(f))
        modes.append(s)

In [19]:
# Normalization of the entrire stack
def entireStackNormalization(stackImgData): 
    # Mean and std of entire stack
    mean = np.mean(stackImgData)
    std = np.std(stackImgData)
    normalizedStackImgData = (stackImgData - mean) / std
    return normalizedStackImgData

In [20]:
# Normalization plane by plane
def planeByPlaneStackNormalization(stackImgData):
    # careful
    mean = np.mean(stackImgData, axis = 0)
    std = np.std(stackImgData, axis = 0)
    normalizedStackImgData = np.array([(stackImgData[i] - mean[i]) / std[i] for i in range(0,np.shape(stackImgData)[0])])
    return normalizedStackImgData

In [21]:
X = np.array([entireStackNormalization(i) for i in imageData])
modes = np.array(modes)

In [22]:
X = np.array(imageData)
Y = onehot_encode(modes)

In [23]:
print(X.shape,Y.shape)

(198, 64, 64, 64) (198, 3)


In [24]:
# data size for conv layer
X_train = np.expand_dims(X,-1)
print(X_train.shape,Y.shape)

(198, 64, 64, 64, 1) (198, 3)


In [27]:
# CNN Model
def get_model2():
    inp = Input(       name='X',     shape=X_train.shape[1:])    
    t   = Conv3D(32,   name='conv1', kernel_size=(3,3,3), activation='relu')(inp)
    #t   = Conv3D(32,   name='conv2', kernel_size=(3,3,3), activation='relu')(t)
    t   = MaxPooling3D(name='pool1', pool_size=(2,2,2))(t)
    #t   = Conv3D(32,   name='conv3', kernel_size=(3,3,3), activation='relu')(t)
    #t   = Conv3D(32,   name='conv4', kernel_size=(3,3,3), activation='relu')(t)
    #t   = MaxPooling3D(name='pool2', pool_size=(2,2,2))(t)
    t   = Flatten(     name='flat')(t)
    #t   = Dense(64,   name='dense', activation='relu')(t)
    oup = Dense(Y.shape[1],    name='Y', activation='softmax')(t)
    model = Model(inp,oup)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

In [28]:
model = get_model2()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
X (InputLayer)               (None, 64, 64, 64, 1)     0         
_________________________________________________________________
conv1 (Conv3D)               (None, 62, 62, 62, 32)    896       
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 31, 31, 31, 32)    0         
_________________________________________________________________
flat (Flatten)               (None, 953312)            0         
_________________________________________________________________
Y (Dense)                    (None, 3)                 2859939   
Total params: 2,860,835
Trainable params: 2,860,835
Non-trainable params: 0
_________________________________________________________________


In [29]:
hist = model.fit(X_train,Y, batch_size=128, epochs=15,validation_split=0.1)

Train on 178 samples, validate on 20 samples
Epoch 1/15
178/178 [==============================] - 471s 3s/step - loss: 10.1286 - acc: 0.3708 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/15
178/178 [==============================] - 499s 3s/step - loss: 11.9527 - acc: 0.2584 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/15
178/178 [==============================] - 478s 3s/step - loss: 11.9527 - acc: 0.2584 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/15
178/178 [==============================] - 467s 3s/step - loss: 11.9527 - acc: 0.2584 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/15
178/178 [==============================] - 471s 3s/step - loss: 11.9527 - acc: 0.2584 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 6/15
178/178 [==============================] - 471s 3s/step - loss: 11.9527 - acc: 0.2584 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 7/15
178/178 [==============================] - 463s 3s/step - loss: 11.9527 - acc: 0.2584 - val_loss: 1.1921e-07 - val_a